In [1]:
## model 다운로드
!pip3 install resnet_pytorch

In [2]:
import os
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import time, datetime
import copy

from torch.optim import lr_scheduler
from torchvision import datasets
from resnet_pytorch import ResNet 
from torch.utils.tensorboard import SummaryWriter

### input param. 지정

In [3]:
batch_size = 32
epochs = 30
input_path = "/home/piai/Documents/Hanja_crop/"
writer = SummaryWriter('./models/runs/exp1/')

### Data load
- input_path 경로 내 train/validation 폴더 지정

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

## train/validation 데이터 resize, normalize
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(input_path, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'validation']}

In [5]:
## 부가 변수 지정
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
class_names = image_datasets['train'].classes 

## GPU 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Training

In [10]:
## model load
model = ResNet.from_pretrained("resnet101", num_classes=len(class_names))
model.to(device)

Loaded pretrained weights for resnet101.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
## param. 지정
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
best_model_weights = copy.deepcopy(model.state_dict())
best_acc = 0.0

In [18]:
## Training
for epoch in range(epochs):
    since = time.time() ## 시간 카운팅 시작(epoch 당 학습 시간 측정)
    print('Epoch {}/{}'.format(epoch+1, epochs))
    print('-' * 10)

    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        ## 학습 weights 위치 지정
        torch.save(model.state_dict(), './models/runs/exp1/weights{}.pth'.format(epoch+1))
        if phase == 'validation' and epoch_acc > best_acc: ## best model 저장
            best_acc = epoch_acc
            best_model_weights = copy.deepcopy(model.state_dict())

    ## 시간 카운팅 종료(epoch 당 학습 시간 측정)
    sec = time.time()- since 
    times = str(datetime.timedelta(seconds = sec)).split(".") 
    times = times[0]
    print("time : ", times)
    print()   

Epoch 1/29
----------
train Loss: 1.8110 Acc: 0.5905
validation Loss: 0.5063 Acc: 0.8552
time :  0:34:46

Epoch 2/29
----------
train Loss: 0.4065 Acc: 0.8805
validation Loss: 0.3509 Acc: 0.8965
time :  0:34:30

Epoch 3/29
----------
train Loss: 0.2694 Acc: 0.9172
validation Loss: 0.2825 Acc: 0.9165
time :  0:34:42

Epoch 4/29
----------
train Loss: 0.2029 Acc: 0.9358
validation Loss: 0.2431 Acc: 0.9286
time :  0:33:12

Epoch 5/29
----------
train Loss: 0.1603 Acc: 0.9477
validation Loss: 0.2446 Acc: 0.9266
time :  0:34:30

Epoch 6/29
----------
train Loss: 0.1314 Acc: 0.9549
validation Loss: 0.2213 Acc: 0.9352
time :  0:34:38

Epoch 7/29
----------
train Loss: 0.0548 Acc: 0.9802
validation Loss: 0.1600 Acc: 0.9520
time :  0:33:56

Epoch 8/29
----------
train Loss: 0.0391 Acc: 0.9857
validation Loss: 0.1645 Acc: 0.9508
time :  0:34:09

Epoch 9/29
----------
train Loss: 0.0328 Acc: 0.9881
validation Loss: 0.1704 Acc: 0.9506
time :  0:33:56

Epoch 10/29
----------
train Loss: 0.0276 Acc:

### best model 

In [19]:
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

torch.save(best_model_weights, './models/runs/exp1/best_weights.pth')

Training complete in 415m 34s
Best val Acc: 0.951972
